# 1) Dataset

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import operator as op
import math
import time

In [2]:
df = pd.read_csv('C:/Users/Jessie/Desktop/MScBA/3. Spring quarter/Mining Big Datasets/Assignment#1/facebook_combined.txt', sep = ' ', header = None)
df.columns = ['node1' , 'node2']
fd = df[['node2', 'node1']]
fd.columns = ['node1' , 'node2']
undir = df.append(fd)
len(df)

88234

# Recommending friends

## 2) Common neighbors: FoF method

In [3]:
def fof(network, nodeID, recommendations = 10):
    
    # Create a list of friends of the node we want to suggest friends to:
    nodes_friends = list(network[network['node1'] == nodeID]['node2'])
    
    # Create a list of nodes that are not friends with the above node:
    non_friends1 = network[network['node1'] != nodeID]
    non_friends2 = non_friends1[non_friends1['node2'] != nodeID]
    non_friends = list(set(non_friends2['node1']).symmetric_difference(set(nodes_friends)))
   
    nonfriends_friends = {}
    nonfriends_count ={}

    for i in range (0, len(non_friends)):
        nonfriends_friends[non_friends[i]] = list(non_friends2[non_friends2['node2'] == non_friends[i]]['node1'])
        nonfriends_count[non_friends[i]] = len(set(nodes_friends).intersection(set(nonfriends_friends[non_friends[i]])))

    sorted_dict1 = sorted(sorted(nonfriends_count.items(), key = op.itemgetter(0)), key = op.itemgetter(1), reverse = True)[:recommendations]
    return([i[0] for i in sorted_dict1])

print(fof(undir, 107))
print(fof(undir, 1126))
print(fof(undir, 14))
print(fof(undir, 35))

[513, 400, 559, 373, 492, 500, 378, 436, 431, 514]
[916, 1238, 1750, 1230, 1004, 1791, 1530, 1172, 1570, 1597]
[2, 17, 140, 111, 137, 162, 19, 333, 44, 243]
[46, 68, 99, 131, 175, 177, 225, 227, 278, 321]


## 3) Jaccard coefficient

In [4]:
def jaccard_coef(network, nodeID, recommendations = 10):
    nodes_friends = list(network[network['node1'] == nodeID]['node2'])
    non_friends1 = network[network['node1'] != nodeID]
    non_friends2 = non_friends1[non_friends1['node2'] != nodeID]
    non_friends = list(set(non_friends2['node1']).symmetric_difference(set(nodes_friends)))
   
    nonfriends_friends = {}
    nonfriends_count = {}

    for i in range (0, len(non_friends)):
        nonfriends_friends[non_friends[i]] = list(non_friends2[non_friends2['node2'] == non_friends[i]]['node1'])
        nonfriends_count[non_friends[i]] = round(len(set(nodes_friends).intersection(set(nonfriends_friends[non_friends[i]]))) / len(set(nodes_friends).union(set(nonfriends_friends[non_friends[i]]))), 3)

    sorted_dict2 = sorted(sorted(nonfriends_count.items(), key = op.itemgetter(0)), key = op.itemgetter(1), reverse = True)[:recommendations]
    return([i[0] for i in sorted_dict2])

print(jaccard_coef(undir, 107))
print(jaccard_coef(undir, 1126))
print(jaccard_coef(undir, 14))
print(jaccard_coef(undir, 35))

[513, 400, 492, 559, 373, 378, 436, 500, 431, 514]
[916, 1750, 1230, 1530, 1004, 1238, 1172, 1791, 1789, 1597]
[2, 140, 17, 162, 111, 333, 44, 137, 19, 243]
[321, 11, 12, 15, 18, 37, 43, 74, 114, 209]


## 4) Adamic & Adar function

In [9]:
def adamic_adar(network, nodeID, recommendations = 10):
    nodes_friends = list(network[network['node1'] == nodeID]['node2'])
    non_friends1 = network[network['node1'] != nodeID]
    non_friends2 = non_friends1[non_friends1['node2'] != nodeID]
    friends_friends = list(set(non_friends2['node1']).symmetric_difference(set(nodes_friends)))

    G = nx.from_pandas_edgelist(network, source = 'node1', target = 'node2')
    pairs = {}

    for i in range(0, len(friends_friends)):
        preds = nx.adamic_adar_index(G, [(nodeID, friends_friends[i])])
        preds_list = list(preds)
        pairs[preds_list[0][1]] = preds_list[0][2]
        
    sorted_dict3 = sorted(sorted(pairs.items(), key = op.itemgetter(0)), key = op.itemgetter(1), reverse = True)[:recommendations]
    return([i[0] for i in sorted_dict3])

print(adamic_adar(undir, 107))
print(adamic_adar(undir, 1126))
print(adamic_adar(undir, 14))
print(adamic_adar(undir, 35))

[513, 400, 559, 500, 492, 373, 378, 436, 524, 514]
[916, 1238, 1750, 1230, 1004, 1791, 1530, 1172, 1570, 1597]
[2, 17, 140, 111, 162, 137, 333, 19, 44, 243]
[46, 68, 99, 131, 175, 177, 225, 227, 278, 321]


## 5) Evaluation of the recommendation system

In [23]:
nodes = np.arange(100, 4039, 100)

# FoF vs Jaccard:
fof_jaccard_similarity = []
for i in range(0, len(nodes)):
    ls_fof = fof(undir, nodes[i])
    ls_jaccard = jaccard_coef(undir, nodes[i])
    
    sim_list = list(set(ls_fof).intersection(set(ls_jaccard)))
    fof_jaccard_similarity.append(10 * len(sim_list))

print("The similarity percentage for each one of the 40 users is \n{}".format(fof_jaccard_similarity))

avg_fof_jaccard = sum(fof_jaccard_similarity)/len(fof_jaccard_similarity)
print("The average similarity between FoF and Jaccard Coefficient is {}%".format(avg_fof_jaccard))

The similarity percetange for each one of the 40 users is 
[20, 90, 60, 80, 90, 10, 30, 90, 70, 60, 80, 0, 50, 20, 70, 50, 10, 100, 10, 70, 70, 90, 40, 10, 90, 100, 30, 90, 20, 70, 60, 50, 70, 60, 60, 0, 20, 50, 90, 90]
The average similarity between FoF and Jaccard Coefficient is 55.5%


In [24]:
# Jaccard vs Adamic & Adar:
jaccard_aa_similarity = []
for i in range(0, len(nodes)):
    ls_jaccard = jaccard_coef(undir, nodes[i])
    ls_aa = adamic_adar(undir, nodes[i])
    
    sim_list = list(set(ls_aa).intersection(set(ls_jaccard)))
    jaccard_aa_similarity.append(10 * len(sim_list))

print("The similarity percentage for each one of the 40 users is \n{}".format(jaccard_aa_similarity))

avg_jaccard_aa = sum(jaccard_aa_similarity)/len(jaccard_aa_similarity)
print("The average similarity between FoF and Jaccard Coefficient is {}%".format(avg_jaccard_aa))

The similarity percetange for each one of the 40 users is 
[30, 80, 60, 80, 80, 10, 30, 90, 70, 70, 80, 30, 40, 20, 70, 50, 10, 100, 20, 70, 60, 90, 40, 20, 90, 100, 30, 90, 30, 70, 60, 50, 80, 60, 70, 0, 20, 50, 90, 90]
The average similarity between FoF and Jaccard Coefficient is 57.0%


In [25]:
# Adamic & Adar vs FoF:
fof_aa_similarity = []
for i in range(0, len(nodes)):
    ls_fof = fof(undir, nodes[i])
    ls_aa = adamic_adar(undir, nodes[i])
    
    sim_list = list(set(ls_aa).intersection(set(ls_fof)))
    fof_aa_similarity.append(10 * len(sim_list))

print("The similarity percentage for each one of the 40 users is \n{}".format(fof_aa_similarity))

avg_fof_aa = sum(fof_aa_similarity)/len(fof_aa_similarity)
print("The average similarity between FoF and Jaccard Coefficient is {}%".format(avg_fof_aa))

The similarity percetange for each one of the 40 users is 
[50, 90, 100, 100, 90, 100, 90, 100, 90, 80, 100, 30, 90, 100, 100, 100, 90, 100, 80, 100, 80, 100, 100, 60, 100, 100, 80, 100, 90, 90, 80, 100, 90, 100, 90, 90, 100, 100, 100, 100]
The average similarity between FoF and Jaccard Coefficient is 90.75%


In [26]:
# Evaluation:
a = [] ; b = [] ; c = [] ; d = [] ; e = [] ; f = [] ; g = [] ; h = []

seed = 0
t1 = time.time()
while len(a) <= 100:
    temp = undir
    remove_real = temp.sample(n = 1, random_state = seed)
    temp = temp.drop(remove_real.index)
    
    f1_fof_list = fof(temp,remove_real.iloc[0,0]) # recommend for f1 with fof method    
    f2_fof_list = fof(temp,remove_real.iloc[0,1]) # recommend for f2 with fof method    
    f1_jaccard_list = jaccard_coef(temp,remove_real.iloc[0,0]) # recommend for f1 with jaccard        
    f2_jaccard_list = jaccard_coef(temp,remove_real.iloc[0,1]) #recommend for f2 with jaccard        
    f1_adam_list = adamic_adar(temp,remove_real.iloc[0,0]) # recommend for f1 with adam method    
    f2_adam_list = adamic_adar(temp,remove_real.iloc[0,1]) #recommend for f2 with adam method
    f1_resal_list = resource_allocation(temp, remove_real.iloc[0,0]) # recommend for f1 with adam method    
    f2_resal_list = resource_allocation(temp, remove_real.iloc[0,1]) #recommend for f2 with adam method
    
    check1_fof = False
    if remove_real.iloc[0, 1] in f1_fof_list:
        check1_fof = True
    
    check2_fof = False
    if remove_real.iloc[0, 0] in f2_fof_list:
        check2_fof = True
    
    check1_jaccard = False
    if remove_real.iloc[0, 1] in f1_jaccard_list:
        check1_jaccard = True
        
    check2_jaccard = False
    if remove_real.iloc[0, 0] in f2_jaccard_list:
        check2_jaccard = True
        
    check1_adam = False
    if remove_real.iloc[0, 1] in f1_adam_list:
        check1_adam = True
        
    check2_adam = False
    if remove_real.iloc[0, 0] in f2_adam_list:
        check2_adam = True 
        
    check1_resal = False
    if remove_real.iloc[0, 1] in f1_resal_list:
        check1_resal = True
        
    check2_resal = False
    if remove_real.iloc[0, 0] in f2_resal_list:
        check2_resal = True
       
    if (check1_fof & check2_fof & check1_jaccard & check2_jaccard & check1_adam & check2_adam & check1_resal & check2_resal):
        a.append(f1_fof_list.index(remove_real.iloc[0,1]))
        b.append(f2_fof_list.index(remove_real.iloc[0,0]))
        c.append(f1_jaccard_list.index(remove_real.iloc[0,1]))
        d.append(f2_jaccard_list.index(remove_real.iloc[0,0]))
        e.append(f1_adam_list.index(remove_real.iloc[0,1]))
        f.append(f2_adam_list.index(remove_real.iloc[0,0]))
        g.append(f1_resal_list.index(remove_real.iloc[0,1]))
        h.append(f2_resal_list.index(remove_real.iloc[0,0]))
     
    seed = seed + 1

In [27]:
fof_avg = (sum(a)/len(a)+sum(b)/len(b))/2
print("Average Rank for FoF is : {}".format(round(fof_avg,2)))

jac_avg = (sum(c)/len(c)+sum(d)/len(d))/2
print("Average Rank for Jaccard Coeff is : {}\n".format(round(jac_avg,2)))

aa_avg = (sum(e)/len(e)+sum(f)/len(f))/2
print("Average Rank for Adaric-Adam is : {}\n".format(round(aa_avg,2)))

resal_avg = (sum(g)/len(g)+sum(h)/len(h))/2
print("Average Rank for resource allocation is : {}\n".format(round(resal_avg,2)))

Average Rank for FoF is : 1.94
Average Rank for Jaccard Coeff is : 2.33

Average Rank for Adaric-Adam is : 1.85

Average Rank for resource allocation is : 1.86



# 6) Assignment Bonus

In [10]:
def resource_allocation(network, nodeID, recommendations = 10):
    nodes_friends = list(network[network['node1'] == nodeID]['node2'])
    non_friends1 = network[network['node1'] != nodeID]
    non_friends2 = non_friends1[non_friends1['node2'] != nodeID]
    friends_friends = list(set(non_friends2['node1']).symmetric_difference(set(nodes_friends)))

    G = nx.from_pandas_edgelist(network, source = 'node1', target = 'node2')
    pairs = {}

    for i in range(0, len(friends_friends)):
        preds = nx.resource_allocation_index(G, [(nodeID, friends_friends[i])])
        preds_list = list(preds)
        pairs[preds_list[0][1]] = preds_list[0][2]

    sorted_dict4 = sorted(sorted(pairs.items(), key = op.itemgetter(0)), key = op.itemgetter(1), reverse = True)[:recommendations]
    return([i[0] for i in sorted_dict4])

print(resource_allocation(undir, 107))
print(resource_allocation(undir, 1126))
print(resource_allocation(undir, 14))
print(resource_allocation(undir, 35))

[3263, 513, 400, 173, 500, 3278, 630, 465, 3290, 559]
[916, 1238, 1750, 1230, 1004, 1791, 1172, 1570, 1530, 1597]
[2, 17, 140, 111, 162, 333, 137, 19, 44, 243]
[46, 68, 99, 131, 175, 177, 225, 227, 278, 321]


In [ ]:
# Evaluation:
a = [] ; b = [] ; c = [] ; d = []

for i in range(0, 100):
    temp = undir
    remove_real = temp.sample(n = 1, random_state = i)
    temp = temp.drop(remove_real.index)
    
    f1_fof_list = fof(temp,remove_real.iloc[0,0]) # recommend for f1 with fof method    
    f2_fof_list = fof(temp,remove_real.iloc[0,1]) # recommend for f2 with fof method    
    f1_jaccard_list = jaccard_coef(temp,remove_real.iloc[0,0]) # recommend for f1 with jaccard        
    f2_jaccard_list = jaccard_coef(temp,remove_real.iloc[0,1]) #recommend for f2 with jaccard        
    f1_adam_list = adamic_adar(temp,remove_real.iloc[0,0]) # recommend for f1 with adam method    
    f2_adam_list = adamic_adar(temp,remove_real.iloc[0,1]) #recommend for f2 with adam method
    f1_resal_list = resource_allocation(temp, remove_real.iloc[0,0]) # recommend for f1 with adam method    
    f2_resal_list = resource_allocation(temp, remove_real.iloc[0,1]) #recommend for f2 with adam method
    
    a.append(1) if remove_real.iloc[0, 1] in f1_fof_list else a.append(0)
    
    b.append(1) if remove_real.iloc[0, 0] in f2_fof_list else b.append(0)
    
    c.append(1) if remove_real.iloc[0, 1] in f1_jaccard_list else c.append(0) 
        
    d.append(1) if remove_real.iloc[0, 0] in f2_jaccard_list else d.append(0)
        
    e.append(1) if remove_real.iloc[0, 1] in f1_adam_list else e.append(0)
        
    f.append(1) if remove_real.iloc[0, 0] in f2_adam_list else f.append(0)
        
    g.append(1) if remove_real.iloc[0, 1] in f1_resal_list else g.append(0)
        
    h.append(1) if remove_real.iloc[0, 0] in f2_resal_list else h.append(0)

In [29]:
print((sum(a)/len(a) + sum(b)/len(b))/2)
print((sum(c)/len(c) + sum(d)/len(d))/2)
print((sum(e)/len(e) + sum(f)/len(f))/2)
print((sum(g)/len(g) + sum(h)/len(h))/2)

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]